In [11]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import os
import pandas as pd
#get data
def GetData(filename):
    dataframe = pd.read_csv(filename)
    dataset = dataframe.values
    Col=len(dataset[0])-1
    X= dataset[:, 0:Col]
    Y=dataset[:, Col]
    return X,Y
#self-train
def self_train(x_lable,y_lable,x_unlable,y_unlable,x_test,y_test):
    #x_test,y_test=x_unlable,y_unlable
    svc=svm.SVC(kernel='rbf',probability=True)
    threshold=0.9
    iteration=0
    test_error=0
    while len(x_unlable)>0:
        svc.fit(x_lable,y_lable)
        add_count=0
        del_index=[]
        #find high confidence point
        for i in range (len(x_unlable)):
            if max(max(svc.predict_proba(x_unlable[i].reshape(1,-1)))) > threshold:
                #add data to lable dataset
                x_lable=np.append(x_lable,[x_unlable[i]],axis=0)
                y_lable=np.append(y_lable,svc.predict([x_unlable[i]]),axis=0)
                del_index.append(i)
                add_count+=1
                #delete label data from unlable dataset
        x_unlable=np.delete(x_unlable,del_index,axis=0)
        y_unlable=np.delete(y_unlable,del_index,axis=0)
        iteration+=1
        if add_count==0:
            svc.fit(x_lable,y_lable)
            test_error=1-svc.score(x_test,y_test)
            break
    return test_error
#kfold algorithm
def Kfold_train(X,y,k):
    kf= KFold(n_splits=k, random_state=50, shuffle=True)
    error_kf=[]
    error_kf_SSL=[]
    for k, (train, test) in enumerate(kf.split(X, y)):
        x_train,x_test=X[train],X[test]
        y_train,y_test=y[train],y[test]
        #fit cad caculate error
        svc=svm.SVC(kernel='poly')
        svc.fit(x_train,y_train)
        error_kf.append(1-svc.score(x_test,y_test))
        #generate lable data
        x_lable,x_unlable,y_lable,y_unlable=train_test_split(x_train,y_train,train_size=0.15,random_state=42)
        #fit cad caculate error
        error_kf_SSL.append(self_train(x_lable,y_lable,x_unlable,y_unlable,x_test,y_test))
        
    return np.mean(error_kf),np.mean(error_kf_SSL)

def Get_KFold_error(path):
    #file path
    fileset= os.listdir(path)
    print "filename     5-fold-error       5-fold-error with self-train"
    for filename in fileset :
        if filename.find(".csv")==-1 :
            continue
        filename1=path+filename
        X,y=GetData(filename1)
        error_kf,error_SSL=Kfold_train(X,y,5)
        print  filename,"         ",error_kf,"       ",error_SSL

path="/Users/xing/Desktop/test1/"
Get_KFold_error(path)
print "-------------------Finish--------------------------"

filename     5-fold-error       5-fold-error with self-train
abalone.csv           0.39846746697991575         0.3716471363493109
credit-approval_test.csv           0.1748677248677249         0.2378306878306878
bank.csv           0.10884955752212391         0.11747787610619473
hepatitis.csv           0.2010752688172043         0.24666666666666667
source_train.csv           0.10038461538461538         0.10038461538461536
energy-y1.csv           0.14203378321025378         0.13942789236906883
car.csv           0.12621596716092817         0.11985926112088466
balance-scale.csv           0.13147096774193548         0.1636258064516129
annealing_train.csv           0.10915880503144652         0.17816037735849052
breast-cancer_train.csv           0.27304347826086955         0.28589371980676326
congressional-voting.csv           0.39633787757284145         0.3848168938786421
target_test.csv           0.04512658227848103         0.02509493670886076
blood.csv           0.2302639821029083         